# ĐỒ ÁN CUỐI KỲ (NMKHDL - CQ2018/2)
### Giảng viên hướng dẫn: Thầy Trần Trung Kiên
### Trợ giảng: Thầy Hồ Xuân Trường

__Thông tin nhóm__: 

* STT: 46

* Họ và tên sinh viên: Nguyễn Hữu Huân - MSSV: 1712466

* Họ và tên sinh viên: Đặng Hữu Thắng - MSSV: 18120555

---

## Import

----

## Thu thập dữ liệu

---

## Khám phá dữ liệu (để xác định câu hỏi)

---

## Đưa ra câu hỏi cần trả lời

---

## Khám phá dữ liệu (để biết tách các tập)

---

## Tiền xử lý (tách các tập)

---

## Khám phá đữ liệu (tập huấn luyện)

---

## Tiền xử lý (tập huấn luyện)

---

## Tiền xử lý (tập validation)

---

## Tiền xử lý + mô hình hóa